In [2]:
import tabula
import sys
import pandas as pd
sys.path.append('../data')

In [3]:
sys.path.append('../scripts/')
from clean_statements import Cleaner 
cleaner= Cleaner()

In [8]:
# convert PDF into CSV
tabula.convert_into("../data/Statement_All_Transactions_20220101_20220701.pdf", "../data/mpesaData.csv", output_format="csv", pages='all')

Got stderr: Sep 03, 2022 6:18:27 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Sep 03, 2022 6:18:27 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [12]:
summary= pd.read_csv('../data/mpesaData.csv', index_col = False,  nrows=11)
summary.drop('Unnamed: 3', axis=1, inplace=True)

In [13]:
summary

,TRANSACTION TYPE,PAID IN,PAID OUT
0,Cash Out,0.00,"7,390.00"
1,Send Money,"52,508.00","64,162.00"
2,Transaction Reversal,50.00,0.00
3,B2C Payment,"46,040.00",0.00
4,Pay Bill,0.00,"39,039.00"
5,Cash In,"51,000.00",0.00
6,FSI Withdraw,"17,000.00",0.00
7,FSI Deposit,0.00,"12,959.00"
8,Customer Merchant Payment,0.00,"39,717.50"
9,Customer Airtime Purchase,0.00,"3,630.00"


In [14]:
summary.to_csv('../data/financialSummary.csv', index=False)

In [15]:
Mpesa_data = pd.read_csv('../data/mpesaData.csv', index_col = False, skiprows=12)
Mpesa_data.drop('Unnamed: 7', axis=1, inplace=True)

In [16]:
Mpesa_data.head()

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
0,QG19OUVM6L,2022-07-01 18:25:24,Customer Transfer to 254798***904 -\rCaroline ...,COMPLETED,0.00,50.00,775.50
1,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill Charge,COMPLETED,0.00,23.00,825.50
2,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill to 200222 - NHIF Acc. 34664143,COMPLETED,0.00,500.00,848.50
3,QG10OHP9A0,2022-07-01 16:02:25,M-Shwari Withdraw,COMPLETED,"1,000.00",0.00,"1,348.50"
4,QG16OF8PA4,2022-07-01 15:31:39,Merchant Payment to 779959 - NAIVAS\rDEVELOPME...,COMPLETED,0.00,120.00,348.50


In [9]:
Mpesa_data['Transaction Status'].value_counts()

COMPLETED             660
Transaction Status     19
Name: Transaction Status, dtype: int64

1. Data Cleaning

In [10]:
Clean_data= cleaner.removerowsByValue(Mpesa_data, column='Transaction Status', val='Transaction Status')

In [11]:
Clean_data['Transaction Status'].value_counts()

COMPLETED    660
Name: Transaction Status, dtype: int64

Drop the Transaction status since all transactions were completed

In [17]:
Mpesa_data = pd.read_csv('../data/mpesaData.csv', index_col = False, skiprows=12)
Mpesa_data.drop('Unnamed: 7', axis=1, inplace=True)

In [31]:
Mpesa_data.tail()

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
674,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
675,QA516GVQ0H,2022-01-05 15:55:39,Buy Bundles,COMPLETED,0.00,20.00,5.00
676,QA5369RQA5,2022-01-05 13:56:44,Buy Bundles,COMPLETED,0.00,50.00,25.00
677,QA4552ABBB,2022-01-04 18:25:46,Merchant Payment to 7899629 -\rDORCAS CHEP...,COMPLETED,0.00,300.00,75.00
678,QA4552782V,2022-01-04 18:24:54,M-Shwari Withdraw,COMPLETED,200.00,0.00,375.00


In [19]:
details=Mpesa_data['Details']

In [20]:
details

0      Customer Transfer to 254798***904 -\rCaroline ...
1                                        Pay Bill Charge
2                Pay Bill to 200222 - NHIF Acc. 34664143
3                                      M-Shwari Withdraw
4      Merchant Payment to 779959 - NAIVAS\rDEVELOPME...
                             ...                        
674                                              Details
675                                          Buy Bundles
676                                          Buy Bundles
677    Merchant Payment to 7899629 -\rDORCAS     CHEP...
678                                    M-Shwari Withdraw
Name: Details, Length: 679, dtype: object

In [35]:
from unicodedata import category


def read_details(details):
    detsA=[]
    
    for row in details:
        if 'Pay Bill Online ' in row:
            category= 'Pay Bill Online'
        elif 'Customer Transfer' in row:
            category = 'Customer Transfer'
        elif 'Merchant Payment 'in row :
            category= 'Merchant Payment'
        elif 'Business Payment' in row:
            category = 'Business Payment' 
        elif 'Funds received' in row:
            category = 'Funds received'
        elif 'Customer Withdrawal' in row:
            category = 'Customer Withdrawal'
        elif 'Deposit of Funds':
            category = 'Deposit of Funds'
        else:
            category= row
        detsA.append(category)
    return detsA
    


In [37]:
details= read_details(details)

In [38]:
Mpesa_data['Details']=details

In [39]:
Mpesa_data.head()

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
0,QG19OUVM6L,2022-07-01 18:25:24,Customer Transfer,COMPLETED,0.00,50.00,775.50
1,QG11OHV8NN,2022-07-01 16:04:31,Deposit of Funds,COMPLETED,0.00,23.00,825.50
2,QG11OHV8NN,2022-07-01 16:04:31,Deposit of Funds,COMPLETED,0.00,500.00,848.50
3,QG10OHP9A0,2022-07-01 16:02:25,Deposit of Funds,COMPLETED,"1,000.00",0.00,"1,348.50"
4,QG16OF8PA4,2022-07-01 15:31:39,Merchant Payment,COMPLETED,0.00,120.00,348.50


In [43]:
Mpesa_data['Details'].value_counts().sum()

679

In [17]:
Data= cleaner.drop_columns(Clean_data,'Transaction Status')

In [18]:
Data.head()

,Receipt No,Completion Time,Details,Paid in,Withdraw\rn,Balance
0,QG19OUVM6L,2022-07-01 18:25:24,Customer Transfer to 254798***904 -\rCaroline ...,0.00,50.00,775.50
1,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill Charge,0.00,23.00,825.50
2,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill to 200222 - NHIF Acc. 34664143,0.00,500.00,848.50
3,QG10OHP9A0,2022-07-01 16:02:25,M-Shwari Withdraw,"1,000.00",0.00,"1,348.50"
4,QG16OF8PA4,2022-07-01 15:31:39,Merchant Payment to 779959 - NAIVAS\rDEVELOPME...,0.00,120.00,348.50


In [19]:
Data.to_csv('../data/TransactionData.csv', index=False)

In [6]:
Data=pd.read_csv('../data/TransactionData.csv')

ParserError: Error tokenizing data. C error: Expected 5 fields in line 5, saw 6


In [21]:
Data.dtypes

Receipt No         object
Completion Time    object
Details            object
Paid in            object
Withdraw\rn        object
Balance            object
dtype: object

Convert columns to the correct data types